In [166]:
from __future__ import print_function
from matplotlib import pyplot as plt

import cudf
# import cuml
import torch

import numpy as np

from timeit import default_timer as timer

In [139]:
torch.cuda.is_available()

True

In [140]:
torch.cuda.get_device_name(0)

'GeForce RTX 2070 SUPER'

In [143]:
torch.cuda.memory_allocated()

512

In [167]:
# hyperparameters to test estimating OLS
N = 100000
k = 4

torch.manual_seed(101010101)

In [168]:
# y is a torch tensor of dimension N, 1
# X is a torch tensor of dimension N, k
# returns bhat (estimate of parameters), Shat (variance-covariance matrix)
def regress(y, X):
    
    # making the estimation easier to read
    Xpy = torch.mm(X.t(),y)
    XpX = torch.mm(X.t(),X)
    
    # parameter estimate (bhat)
    bhat, LU = torch.solve(Xpy, XpX)
    
    # calculating the variance-covariance matrix (Shat)
    ehat = y - torch.mm(X, bhat)
    shat2 = torch.mm(ehat.t(), ehat)/(N-k)
    Shat = (XpX.inverse()*shat2)
    
    return bhat, Shat

In [175]:
# CUDA version

# randomly creating b (beta)
b = torch.cat((2*torch.rand(k-1,1)-0.5, torch.zeros(1,1) + 1), 0).cuda()

# randomly creating X and y from b
Xnocons = torch.rand(N, k-1).cuda()
cons = (torch.zeros(N,1) + 1).cuda()
X = torch.cat((Xnocons, cons), 1)
e = (torch.rand(N,1) - 0.5).cuda() # rand is between [0,1)
y = torch.mm(X, b) + e

start = timer()
bhat, Shat = regress(y, X)
end = timer()

print(end - start)

0.005583880003541708


In [181]:
# CPU version

# randomly creating b (beta)
b = torch.cat((2*torch.rand(k-1,1)-0.5, torch.zeros(1,1) + 1), 0)

# randomly creating X and y from b
Xnocons = torch.rand(N, k-1)
cons = (torch.zeros(N,1) + 1)
X = torch.cat((Xnocons, cons), 1)
e = (torch.rand(N,1) - 0.5) # rand is between [0,1)
y = torch.mm(X, b) + e

start = timer()
bhat, Shat = regress(y, X)
end = timer()

print(end - start)

0.0019474120053928345


In [160]:
b

tensor([[0.7450],
        [0.5557],
        [0.0084],
        [1.0000]], device='cuda:0')

In [161]:
bhat

tensor([[0.7472],
        [0.5569],
        [0.0106],
        [0.9981]], device='cuda:0')

In [162]:
Shat

tensor([[ 9.9210e-06, -8.9410e-08,  3.5537e-08, -4.9240e-06],
        [-8.9407e-08,  9.9326e-06,  4.7515e-08, -4.9468e-06],
        [ 3.5536e-08,  4.7512e-08,  9.9463e-06, -5.0162e-06],
        [-4.9240e-06, -4.9468e-06, -5.0162e-06,  8.2675e-06]], device='cuda:0')

In [163]:
sbhat = np.apply_along_axis(np.sqrt, 0, S.diag().numpy())

In [164]:
sbhat

array([0.0031719 , 0.00316232, 0.00316091, 0.00289191], dtype=float32)

In [165]:
torch.cuda.memory_allocated()

4004352